# Estratégias de pesquisa adversarial e árvores de decisão

**Trabalho desenvolvido por:** Afonso Coelho, Diogo Azevedo e Miguel Carvalho

# 1.0 Introdução

## 1.1 Contexto
O notebook que se segue foi desenvolvido no âmbito da cadeira de Inteligência Artificial (FCUP-CC2006) da Licenciatura de Inteligência Artifical (L:IACD). Trata-se de um trabalho no qual se propõe implementar um modelo de Monte Carlo Tree Search (MCTS), aplicá-lo ao famoso jogo de tabuleiro 4 Em Linha (Connect 4) e implementar e treinar uma árvore de decisão baseada num *dataset* gerado por vários jogos entre AIs de MCTS. 

## 1.2 Imports
Como seria de esperar, ao longo do desenvolvimento foi necessário importar certas bibliotecas. Eis o seu nome e a sua função:  
- **os** e **sys:**  
    Usados para manipulação de caminhos de ficheiros, diretórios e integração com o sistema operativo, como leitura/escrita de ficheiros e configuração do ambiente de execução.

- **pandas:**  
    Utilizado para manipulação, análise e processamento de dados tabulares, especialmente para carregar, transformar e guardar datasets de jogos e resultados.

- **numpy:**  
    Usado para operações matemáticas e manipulação eficiente de arrays, essencial para cálculos numéricos e representação do estado do tabuleiro.

- **matplotlib.pyplot:**  
    Utilizado para criar gráficos e visualizações, como análise de resultados, evolução de métricas e visualização de árvores.

- **seaborn:**  
    Biblioteca complementar ao matplotlib para criar gráficos estatísticos mais avançados e visualmente apelativos, como heatmaps e distribuições.

- **networkx:**  
    Usado para criar, manipular e visualizar grafos, nomeadamente para representar e desenhar a árvore de pesquisa do MCTS.

- **concurrent.futures:**  
    Utilizado para paralelizar tarefas, como simulações de jogos ou execuções de múltiplos MCTS em simultâneo, acelerando o processamento.

- **sklearn:**  
    Usado para avaliação da árvore de decisão, nomeadamente e exclusivamente para como para dividir datasets e calcular métricas de desempenho.  
    <u>Esta biblioteca não foi utilizada de todo no sentido da implementação do modelo ou do treino do mesmo.</u>

Foram ainda importadas algumas *features* nativas ao python, como por exemplo:  
- **math:** Usado para cálculos matemáticos.

- **random:** Usado para calcular números aleatórios ou pseudoaleatórios.

- **csv:** Usado para a leitura do *dataset*, guardado em formato de csv.


## 1.3 4 Em Linha: Regras de jogo
### 1.3.1 Elementos de jogo
- Tabuleiro de 7 colunas por 6 linhas
- 21 peças vermelhas (ou outra cor)
- 21 peças amarelas (ou outra cor)

### 1.3.2 Objetivo
Ser o primeiro jogador a formar um "4 em linha" isto é, 4 peças adjacentes da mesma cor formarem uma linha horizontal, vertical ou diagonal.

### 1.3.3 Como jogar
Primeiramente decide-se por acordo quem joga primeiro. Daí basta seguir os passos abaixo:

1. O jogador A escolhe uma coluna para largar a sua peça.
2. O jogador B escolhe uma coluna para largar a sua peça.
3. Assim, seguem, trocando de vez a cada jogada, até que um "4 em linha" seja formado.

## 1.3.4 Regras
Na versão física do jogo, o tabuleiro é colocado de pé e por isso é afetado pela gravidade. Implementando esse jogo digitalmente significa que:
- Uma peça largada fica colocada na primeira casa (a contar de baixo para cima) tal que abaixo dela todas as casas abaixo dela estejam ocupadas, ou que seja o fim do tabuleiro.

## 1.4 Implementação do jogo
A implementação regras do está separada ao longo de alguns ficheiros por motivos de boas práticas, nomeadamente na classe [ConnectFour](Game/ConnectFour.py) que retira algumas constantes como o tamanho do tabuleiro de um ficheiro de [configuração](utils/config.py). Esta classe realiza as tarefas mais importantes a nível de lógica de jogo, como:

1. Guardar o estado do jogo, como o tabuleiro (em forma de matrix `numpy`), de quem é a vez de jogar e quantas peças estão no tabuleiro:

In [ ]:
import utils.config as config
import numpy as np

def __init__(self) -> None:
        """
        Create a new game.
        """
        self.turn = 1
        self.win = 0
        self.board = np.zeros((config.ROW, config.COLUMN), dtype=np.int8)
        self.last_move = []
        self.pieces = 0

2. Verificar se um lance é válido, correspondendo a tentativa com a lista de lances válidos:

In [ ]:
import utils.config as config
import numpy as np

def legal_moves(self) -> list:
    """
    Return the legal moves.
    Returns
    -------
    legal_moves: a list of legal moves
    """
    return [i for i in range(config.COLUMN) if self.board[0][i] == 0]


3. Verificar se uma posição do tabuleiro representa vitória para algum jogador:

In [ ]:
import utils.config as config
import numpy as np

def check_win(self) -> int:
    """
    Check if the game is won.
    Returns
    -------
    win: 1 if player 1 wins, -1 if player 2 wins, 0 otherwise
    """
    # Check horizontal
    for i in range(config.ROW):
        for j in range(config.COLUMN - 3):
            if (
                self.board[i, j]
                == self.board[i, j + 1]
                == self.board[i, j + 2]
                == self.board[i, j + 3]
                != 0
            ):
                return self.board[i, j]
    # Check vertical
    for i in range(config.ROW - 3):
        for j in range(config.COLUMN):
            if (
                self.board[i, j]
                == self.board[i + 1, j]
                == self.board[i + 2, j]
                == self.board[i + 3, j]
                != 0
            ):
                return self.board[i, j]
    # Check diagonal
    for i in range(config.ROW - 3):
        for j in range(config.COLUMN - 3):
            if (
                self.board[i, j]
                == self.board[i + 1, j + 1]
                == self.board[i + 2, j + 2]
                == self.board[i + 3, j + 3]
                != 0
            ):
                return self.board[i, j]
    for i in range(config.ROW - 3):
        for j in range(config.ROW - 3, config.COLUMN):
            if (
                self.board[i, j]
                == self.board[i + 1, j - 1]
                == self.board[i + 2, j - 2]
                == self.board[i + 3, j - 3]
                != 0
            ):
                return self.board[i, j]
    return 0


4. E realizar uma jogada:

In [ ]:
import utils.config as config
import numpy as np

def play(self, move: int) -> int:
    """
    Play a move.
    Parameters
    ----------
    move: the move to play
    Returns
    -------
    win: 1 if player 1 wins, -1 if player 2 wins, 0 otherwise
    """
    if move not in self.legal_moves():
        raise ValueError("Illegal move")
    for i in range(config.ROW - 1, -1, -1):
        if self.board[i][move] == 0:
            self.board[i][move] = self.turn
            self.last_move = [i, move]
            break
    self.turn *= -1
    self.win = self.check_win()
    self.pieces += 1
    return self.win

Entre outras tarefas, cuja implementação pode ser encontrada clicando [aqui](Game/ConnectFour.py) ou no bloco de código colapsável abaixo.

##### ConnectFour.py

In [ ]:
import utils.config as config
import numpy as np


class ConnectFour(object):
    """
    A class used to represent a game of Connect Four.

    Methods
    -------
    reset_game() -> None
        Reset the game.
    copy() -> copy
        Return a copy of the game.
    check_win() -> int
        Check if the game is won.
    legal_moves() -> list
        Return the legal moves.
    play(move: int) -> int
        Play a move.
    is_over() -> bool
        Check if the game is over.
    print_board() -> None
        Print the board.
    """

    def __init__(self) -> None:
        """
        Create a new game.
        """
        self.turn = 1
        self.win = 0
        self.board = np.zeros((config.ROW, config.COLUMN), dtype=np.int8)
        self.last_move = []
        self.pieces = 0

    def reset_game(self) -> None:
        """
        Reset the game.

        Returns
        -------
        none
        """
        self.turn = 1
        self.win = 0
        self.board = np.zeros((config.ROW, config.COLUMN), dtype=np.int8)
        self.last_move = []

    def copy(self) -> "ConnectFour":
        """
        Return a copy of the game.

        Returns
        -------
        copy: a copy of the game
        """
        new_game = ConnectFour()
        new_game.turn = self.turn
        new_game.win = self.win
        new_game.board = self.board.copy()
        new_game.last_move = self.last_move[:]
        new_game.pieces = self.pieces
        return new_game

    def check_win(self) -> int:
        """
        Check if the game is won.

        Returns
        -------
        win: 1 if player 1 wins, -1 if player 2 wins, 0 otherwise
        """
        # Check horizontal
        for i in range(config.ROW):
            for j in range(config.COLUMN - 3):
                if (
                    self.board[i, j]
                    == self.board[i, j + 1]
                    == self.board[i, j + 2]
                    == self.board[i, j + 3]
                    != 0
                ):
                    return self.board[i, j]
        # Check vertical
        for i in range(config.ROW - 3):
            for j in range(config.COLUMN):
                if (
                    self.board[i, j]
                    == self.board[i + 1, j]
                    == self.board[i + 2, j]
                    == self.board[i + 3, j]
                    != 0
                ):
                    return self.board[i, j]
        # Check diagonal
        for i in range(config.ROW - 3):
            for j in range(config.COLUMN - 3):
                if (
                    self.board[i, j]
                    == self.board[i + 1, j + 1]
                    == self.board[i + 2, j + 2]
                    == self.board[i + 3, j + 3]
                    != 0
                ):
                    return self.board[i, j]
        for i in range(config.ROW - 3):
            for j in range(config.ROW - 3, config.COLUMN):
                if (
                    self.board[i, j]
                    == self.board[i + 1, j - 1]
                    == self.board[i + 2, j - 2]
                    == self.board[i + 3, j - 3]
                    != 0
                ):
                    return self.board[i, j]
        return 0

    def legal_moves(self) -> list:
        """
        Return the legal moves.

        Returns
        -------
        legal_moves: a list of legal moves
        """
        return [i for i in range(config.COLUMN) if self.board[0][i] == 0]

    def play(self, move: int) -> int:
        """
        Play a move.

        Parameters
        ----------
        move: the move to play

        Returns
        -------
        win: 1 if player 1 wins, -1 if player 2 wins, 0 otherwise
        """
        if move not in self.legal_moves():
            raise ValueError("Illegal move")
        for i in range(config.ROW - 1, -1, -1):
            if self.board[i][move] == 0:
                self.board[i][move] = self.turn
                self.last_move = [i, move]
                break
        self.turn *= -1
        self.win = self.check_win()
        self.pieces += 1
        return self.win

    def is_over(self) -> bool:
        """
        Check if the game is over.

        Returns
        -------
        bool: True if the game is over, False otherwise
        """
        return self.win != 0 or len(self.legal_moves()) == 0

    def print_board(self) -> None:
        """
        Print the board.

        Returns
        -------
        none
        """
        print(self.board)